<a href="https://colab.research.google.com/github/quang-m-nguyen/DeepPGD/blob/main/dna_bert2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
from transformers.models.bert.configuration_bert import BertConfig
import torch
from transformers import AutoTokenizer, AutoModel

config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/root/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def get_embedding(dna):
  inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
  hidden_states = model(inputs)[0] # [1, sequence_length, 768]
  embedding_mean = torch.mean(hidden_states[0], dim=0)
  return embedding_mean

dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
embedding = get_embedding(dna)
print(len(embedding))

In [38]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv', delimiter='\t')
df_limited = df.head(5)
display(df_limited)

,index,label,text
0,0,1,ATTAACAGTTTCCATAAAATCGGGACTAGCTGTCCAAAAAT
1,1,1,GAGAATAAGACTTATTCTCTCAGCAAGTAGTTTTGAGATCA
2,2,1,ATGTAGAGACAAACTACCTACAGCAGTAGTGGTTTCTCGTT
3,3,1,ATTTTTACCATCATAACATTCGTTATCAGTGGTCTTGTGGA
4,4,1,ATACAAGTATCACTTATATACAGAATTTCAAAGAGACCGTT


In [36]:
import pickle
import os
import pandas as pd

def process_tsv_and_save_embeddings(tsv_file_path):
    # Read the TSV file
    df = pd.read_csv(tsv_file_path, delimiter='\t')
    embeddings_data = []
    count = 0

    # Process each DNA sequence
    for index, row in df.iterrows():
        dna = row['text']
        embedding = get_embedding(dna)
        embeddings_data.append({'dna': dna, 'embedding': embedding})
        count += 1
        print(f"Finished processing DNA sequence {count}: {dna}, embedding shape: {embedding.shape}")

    # Determine the output file path
    tsv_dir = os.path.dirname(tsv_file_path)
    tsv_filename = os.path.basename(tsv_file_path).replace('.tsv', '')
    output_file_path = os.path.join(tsv_dir, f"{tsv_filename}_embeddings_data.pkl")

    # Save the embeddings data as a pickle file
    with open(output_file_path, 'wb') as f:
        pickle.dump(embeddings_data, f)

    print(f"Embeddings data saved to {output_file_path}")

# Example usage
tsv_file_path = '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test.tsv'
process_tsv_and_save_embeddings(tsv_file_path)

Finished processing DNA sequence 1: GCTGGCCTCCCGTGCGCGGTCCCGCGCGCGGTTGGCCCAAA, embedding shape: torch.Size([768])
Finished processing DNA sequence 2: ACTGCGGCATTTGGGCCAACCGCGCGCGGGACCGCGCACGG, embedding shape: torch.Size([768])
Finished processing DNA sequence 3: GAACCGAGCGCCACGACAATCGGTGTGGTTGAGAGACCTCG, embedding shape: torch.Size([768])
Finished processing DNA sequence 4: TGAGAGACCTCGCGTCCCGTCGTGCGCGCCTCGTCGCCCCG, embedding shape: torch.Size([768])
Finished processing DNA sequence 5: CGTCGTGCGCGCCTCGTCGCCCCGAGCGGTGGAAAGACCCT, embedding shape: torch.Size([768])
Finished processing DNA sequence 6: AAAGAAACTTACGAGGATTCCCCTAGTAACGGCGAGCGAAC, embedding shape: torch.Size([768])
Finished processing DNA sequence 7: GCTCTTCCCGGTTCGCTCGCCGTTACTAGGGGAATCCTCGT, embedding shape: torch.Size([768])
Finished processing DNA sequence 8: CCAGCTTGAGAATCGGGCGCCCCTGGCGTCCGAATTGTAGT, embedding shape: torch.Size([768])
Finished processing DNA sequence 9: CGAATTGTAGTCTGGAGAAGCGTCCTCAGCGGCGGACCGGG, embedding

In [ ]:
# List of TSV file paths to process
tsv_file_paths = [
    '/content/drive/MyDrive/deepPGD/6mA/6mA_Xoc BLS256/test.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_D.melanogaster/test.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_A.thaliana/test.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_H.sapiens/test.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_C.elegans/test.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_T.thermophile/test.tsv',
    '/content/drive/MyDrive/deepPGD/5hmC/5hmC_H.sapiens/test.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_S.cerevisiae/test.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/test.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_Xoc BLS256/train.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_D.melanogaster/train.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_A.thaliana/train.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_H.sapiens/train.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_C.elegans/train.tsv',
    '/content/drive/MyDrive/deepPGD/6mA/6mA_T.thermophile/train.tsv',
    '/content/drive/MyDrive/deepPGD/5hmC/5hmC_H.sapiens/train.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_S.cerevisiae/train.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv'
]

# Loop through each file path and process it
for tsv_file_path in tsv_file_paths:
    process_tsv_and_save_embeddings(tsv_file_path)

Finished processing DNA sequence 1: TGATGGACCAGATGCGAAGCAACCGCTTCCAGTCTGCCTGG, embedding shape: torch.Size([768])
Finished processing DNA sequence 2: GCTTTTCCCAGCTTCCGCGGAACGCTTCGCTCCAATCGAGT, embedding shape: torch.Size([768])
Finished processing DNA sequence 3: GCGTTCCGCGGAAGCTGGGAAAAGCAATGGCGAGCAAGATC, embedding shape: torch.Size([768])
Finished processing DNA sequence 4: TGATCTCGGGATCGCCGTGGATTTCGAAGGGGCCGAATTGC, embedding shape: torch.Size([768])
Finished processing DNA sequence 5: CAACAGGATGCCGAGCAGGTAGAGAATTTCTTCTGCAGTGC, embedding shape: torch.Size([768])
Finished processing DNA sequence 6: AATCTGGTCGTGTGCTGGGGAGGGCATTCCATTTTGCGCGA, embedding shape: torch.Size([768])
Finished processing DNA sequence 7: AACTTGGTGGCATAGTCACCAAAGAAGTTGTGCTTCCACTG, embedding shape: torch.Size([768])
Finished processing DNA sequence 8: CGGCACCTTGCAGTGGAAGCACAACTTCTTTGGTGACTATG, embedding shape: torch.Size([768])
Finished processing DNA sequence 9: TTGGACAACCACACAGAAGCAGCAGCACCCTGGAACAGACC, embedding

In [37]:
import pickle

def read_embeddings_data(pkl_file_path, num_rows=5):
    with open(pkl_file_path, 'rb') as f:
        embeddings_data = pickle.load(f)

    limited_data = embeddings_data[:num_rows]
    for item in limited_data:
        print(f"DNA: {item['dna']}, Embedding shape: {item['embedding'].shape}")

# Example usage
pkl_file_path = '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test_embeddings_data.pkl'
read_embeddings_data(pkl_file_path)

DNA: GCTGGCCTCCCGTGCGCGGTCCCGCGCGCGGTTGGCCCAAA, Embedding shape: torch.Size([768])
DNA: ACTGCGGCATTTGGGCCAACCGCGCGCGGGACCGCGCACGG, Embedding shape: torch.Size([768])
DNA: GAACCGAGCGCCACGACAATCGGTGTGGTTGAGAGACCTCG, Embedding shape: torch.Size([768])
DNA: TGAGAGACCTCGCGTCCCGTCGTGCGCGCCTCGTCGCCCCG, Embedding shape: torch.Size([768])
DNA: CGTCGTGCGCGCCTCGTCGCCCCGAGCGGTGGAAAGACCCT, Embedding shape: torch.Size([768])


/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
